In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.neighbors import KNeighborsClassifier
import time

In [2]:
feature_list = ['x','y','hour','weekday','day','month','year', 'accuracy']

In [3]:
def prepare_data(df, n_cell_x, n_cell_y):
    #Creating the grid
    size_x = 10. / n_cell_x
    size_y = 10. / n_cell_y
    eps = 0.00001  
    xs = np.where(df.x.values < eps, 0, df.x.values - eps)
    ys = np.where(df.y.values < eps, 0, df.y.values - eps)
    pos_x = (xs / size_x).astype(np.int)
    pos_y = (ys / size_y).astype(np.int)
    df['grid_x'] = pos_x
    df['grid_y'] = pos_y
    
    #Feature engineering
    fw = [400, 1000, 4, 3.5, 1./22., 2, 9, 0.09] #feature weights (black magic here)
    df.x = df.x.values * fw[0]
    df.y = df.y.values * fw[1]
    initial_date = np.datetime64('2014-01-01T01:01', dtype='datetime64[m]') 
    d_times = pd.DatetimeIndex(initial_date + np.timedelta64(int(mn), 'm') 
                               for mn in df.time.values)    
    df['hour'] = (d_times.hour+ d_times.minute/60) * fw[2]
    df['weekday'] = d_times.weekday * fw[3]
    df['day'] = (d_times.dayofyear * fw[4]).astype(int)
    df['month'] = d_times.month * fw[5]
    df['year'] = (d_times.year - 2013) * fw[6]
    df.accuracy = df.accuracy.values * fw[7]
    df = df.drop(['time'], axis=1)
    
    return df

In [4]:
def run_prediction(df_train, df_test, n_cell_x, n_cell_y, th):
    total_result = pd.DataFrame()
    for x_grid in range(0, n_cell_x):
        start_time = time.time()
        for y_grid in range(0, n_cell_y):
            total_result = total_result.append(process_one_cell(df_train, df_test, x_grid, y_grid, th))
        print("Elapsed time overall: %s seconds" % (time.time() - start_time), x_grid, flush = True)
    return total_result

In [5]:
def process_one_cell(df_train, df_test, grid_x, grid_y, th):
    start_time = time.time()
    
    df_cell_train = df_train[((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x + 1)&(df_train.grid_y == grid_y - 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y + 1)) |
                             ((df_train.grid_x == grid_x - 1)&(df_train.grid_y == grid_y - 1))]
    place_counts = df_cell_train.place_id.value_counts()
    mask = (place_counts[df_cell_train.place_id.values] >= th).values
    df_cell_train = df_cell_train.loc[mask]
    
    df_cell_test = df_test[(df_test.grid_x == grid_x)&(df_test.grid_y == grid_y)]
    row_ids = df_cell_test.row_id
    
    #Applying the classifier
    clf = KNeighborsClassifier(n_neighbors=35, weights='distance', 
                               metric='manhattan')
    clf.fit(df_cell_train[feature_list], df_cell_train.place_id)
    predictions = clf.predict_proba(df_cell_test[feature_list])
    result_index = np.argsort(predictions, axis=1)[:,::-1][:,:3]
    result = pd.DataFrame(df_cell_test.row_id)
    result['p1'] = clf.classes_[result_index][:,:1]
    result['p2'] = clf.classes_[result_index][:,1:2]
    result['p3'] = clf.classes_[result_index][:,2:3]
    
    return result

In [6]:
# Run validation
df_train = pd.read_csv('../train.csv',
                           usecols=['row_id','x','y','accuracy','time','place_id'])

df_valiation = df_train[df_train.time > 786239 * 0.875]

df_train = df_train[df_train.time <= 786239 * 0.875]

n_cell_x = 30
n_cell_y = 60 

print('Preparing train data')
df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_valiation = prepare_data(df_valiation, n_cell_x, n_cell_y)
df_valiation['p1'] = np.nan
df_valiation['p2'] = np.nan
df_valiation['p3'] = np.nan

prediction_result = run_prediction(df_train, df_valiation, n_cell_x, n_cell_y, 5)


Preparing train data
Elapsed time overall: 106.98082089424133 seconds 0
Elapsed time overall: 118.86453318595886 seconds 1
Elapsed time overall: 117.29410934448242 seconds 2
Elapsed time overall: 116.85728001594543 seconds 3
Elapsed time overall: 118.99973654747009 seconds 4
Elapsed time overall: 121.4804277420044 seconds 5
Elapsed time overall: 123.10025262832642 seconds 6
Elapsed time overall: 121.18829846382141 seconds 7
Elapsed time overall: 131.77376413345337 seconds 8
Elapsed time overall: 139.89727544784546 seconds 9
Elapsed time overall: 126.47413969039917 seconds 10
Elapsed time overall: 122.64227390289307 seconds 11
Elapsed time overall: 135.5942394733429 seconds 12
Elapsed time overall: 135.70360040664673 seconds 13
Elapsed time overall: 123.13283133506775 seconds 14
Elapsed time overall: 124.91680526733398 seconds 15
Elapsed time overall: 122.79931306838989 seconds 16
Elapsed time overall: 122.99077367782593 seconds 17
Elapsed time overall: 131.9342110157013 seconds 18
Elap

In [7]:
# Calculate score
prediction_result.sort_index(inplace=True)
prediction_result['score'] = (prediction_result.p1 == df_valiation.place_id) * 1
prediction_result['score'] += (prediction_result.p2 == df_valiation.place_id) * 0.5
prediction_result['score'] += (prediction_result.p3 == df_valiation.place_id) * 0.33

prediction_result.score.describe()

count    4417684.000000
mean           0.540568
std            0.451199
min            0.000000
25%            0.000000
50%            0.500000
75%            1.000000
max            1.000000
Name: score, dtype: float64

Validation result:

In [8]:
# Write to file
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_validation_result.csv', index=False)

In [9]:
# Run test
df_train = pd.read_csv('../train.csv',
                       usecols=['row_id','x','y','accuracy','time','place_id'])
df_test = pd.read_csv('../test.csv',
                       usecols=['row_id','x','y','accuracy','time'])
n_cell_x = 30
n_cell_y = 60 

df_train = prepare_data(df_train, n_cell_x, n_cell_y)
df_test = prepare_data(df_test, n_cell_x, n_cell_y)
df_test['p1'] = np.nan
df_test['p2'] = np.nan
df_test['p3'] = np.nan
prediction_result = run_prediction(df_train, df_test, 5)

prediction_result.sort_index(inplace=True)
prediction_result['place_id'] = prediction_result.p1.astype(str) + " " + \
                                 prediction_result.p2.astype(str) + " " + \
                                 prediction_result.p3.astype(str)
prediction_result[['row_id', 'place_id']].to_csv('new_baseline_result.csv', index=False)

Elapsed time overall: 176.69736409187317 seconds 0
Elapsed time overall: 182.33909511566162 seconds 1
Elapsed time overall: 179.95632672309875 seconds 2
Elapsed time overall: 194.7431447505951 seconds 3
Elapsed time overall: 195.13991928100586 seconds 4
Elapsed time overall: 194.94174075126648 seconds 5
Elapsed time overall: 202.69043922424316 seconds 6
Elapsed time overall: 201.27015614509583 seconds 7
Elapsed time overall: 199.84535193443298 seconds 8
Elapsed time overall: 188.66247296333313 seconds 9
Elapsed time overall: 187.3608193397522 seconds 10
Elapsed time overall: 210.6493799686432 seconds 11
Elapsed time overall: 188.98353624343872 seconds 12
Elapsed time overall: 209.71220684051514 seconds 13
Elapsed time overall: 198.13238406181335 seconds 14
Elapsed time overall: 194.97281408309937 seconds 15
Elapsed time overall: 191.00574207305908 seconds 16
Elapsed time overall: 196.04477214813232 seconds 17
Elapsed time overall: 206.2481746673584 seconds 18
Elapsed time overall: 192.